In [1]:
# loading in required libraries
import numpy as np
import pandas as pd
import os
import glob
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import shap

/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# reading in skater data
skaters_21 = pd.read_csv('skaters_22.csv')
skaters_22 = pd.read_csv('skaters_23.csv')
skaters_23 = pd.read_csv('skaters_24.csv')
skaters_24 = pd.read_csv('skaters_25.csv')

In [5]:
# reading in team and player GAR data as well as power play data
player_gar = pd.read_excel('gar_stats.xlsx', sheet_name = 'GAR by Tm', header = 1)
team_gar = pd.read_excel('gar_stats.xlsx', sheet_name = 'Tm GAR', header = 1)
pp_percent = pd.read_excel('gar_stats.xlsx', sheet_name = 'Tm Stats')

/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/ml_env/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [6]:
# filtering gar data
filtered = ((player_gar['year_ID'] > 2021) & (player_gar['pos'] != 'G'))
player_gar = player_gar[filtered]
team_gar = team_gar[team_gar['Year'] > 2021]

In [8]:
# row binding all the skater data
combined_skaters = pd.concat([skaters_21, skaters_22, skaters_23, skaters_24], ignore_index = True)
combined_skaters['season'] = combined_skaters['season'] + 1

In [61]:
# selecting reasonable variables to include based on domain knowledge
skater_vars = ['season', 'name', 'team', 'situation', 'games_played', 'icetime', 'I_F_xGoals',
 'I_F_xRebounds', 'I_F_shotsOnGoal',  'I_F_goals', 'I_F_rebounds', 'I_F_reboundGoals',
 'I_F_lowDangerShots', 'I_F_mediumDangerShots','I_F_highDangerShots', 'I_F_lowDangerGoals',
 'I_F_mediumDangerGoals','I_F_highDangerGoals', 'I_F_oZoneShiftStarts']
reduced_skaters = combined_skaters[skater_vars]

In [62]:
# filtering power play data
pp_percent = pp_percent[pp_percent['year_ID'] > 2021]
pp_vars = ['year_ID', 'team1', 'PP%']
pp_percent = pp_percent[pp_vars]

KeyError: 'year_ID'

In [11]:
# filter player gar data
player_gar_vars = ['player_name', 'year_ID', 'age', 'team_ID', 'pos', 'GP', 'PPG', 'S%', 'OFF', 'GAR']
reduced_pgar = player_gar[player_gar_vars]

In [12]:
# organized and filtering team gar data
team_gar.columns.tolist()
tgar_vars = ['Year', 'Tm', 'Off.', 'Total']
reduced_tgar = team_gar[tgar_vars]
reduced_tgar.rename(columns = {'Off.':'Offensive GAR', 'Total':'Total GAR'}, inplace = True)
reduced_tgar.rename(columns = {'Tm':'Team'}, inplace = True)

/var/folders/ws/n_fmk_5j2xn553m9_3d3glwc0000gn/T/ipykernel_80558/998210531.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_tgar.rename(columns = {'Off.':'Offensive GAR', 'Total':'Total GAR'}, inplace = True)
/var/folders/ws/n_fmk_5j2xn553m9_3d3glwc0000gn/T/ipykernel_80558/998210531.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_tgar.rename(columns = {'Tm':'Team'}, inplace = True)


In [63]:
# extracting avg power play ice time for all players and adjusting counting stats to per 60 min rate
pp_filter = reduced_skaters['situation'] == '5on4'
reduced_skaters.loc[pp_filter, 'avg_pp_toi'] = (
    reduced_skaters.loc[pp_filter, 'icetime']
) / reduced_skaters.loc[pp_filter, 'games_played']


reduced_skaters['avg_pp_toi'] = (
    reduced_skaters
        .groupby(['name', 'season'])['avg_pp_toi']
        .transform('first')          
)
per60_cols = [
    'I_F_xGoals', 'I_F_xRebounds', 'I_F_shotsOnGoal', 'I_F_goals',
    'I_F_rebounds', 'I_F_reboundGoals', 'I_F_lowDangerShots',
    'I_F_mediumDangerShots', 'I_F_highDangerShots',
    'I_F_lowDangerGoals', 'I_F_mediumDangerGoals', 'I_F_highDangerGoals'
]

filtered = (reduced_skaters['situation'] == 'all') & (reduced_skaters['icetime'] > 29940)

# Apply the per-60 transformation to selected rows and columns
reduced_skaters.loc[filtered, per60_cols] = (
    reduced_skaters.loc[filtered, per60_cols]
    .div(reduced_skaters.loc[filtered, 'icetime'], axis=0)
    .mul(3600)
)                                                                   
reduced_skaters = reduced_skaters[filtered]

/var/folders/ws/n_fmk_5j2xn553m9_3d3glwc0000gn/T/ipykernel_80558/481354738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_skaters.loc[pp_filter, 'avg_pp_toi'] = (
/var/folders/ws/n_fmk_5j2xn553m9_3d3glwc0000gn/T/ipykernel_80558/481354738.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_skaters['avg_pp_toi'] = (


In [17]:
# renaming and organizing datasets then creating one big players dataset with gar data
pp_percent.rename(columns = {'year_ID':'Year', 'team1':'Team'}, inplace = True)
reduced_pgar.rename(columns = {'year_ID':'Year', 'player_name':'name'}, inplace = True)
team_stats = pd.merge(pp_percent, reduced_tgar, how = 'inner', on = ['Year', 'Team'])
team_stats.rename(columns = {'Offensive GRA':'Offensive GAR', 'Total GRA':'Total GAR'}, inplace = True)
reduced_skaters.rename(columns = {'season':'Year'}, inplace = True)

player_stats = pd.merge(reduced_skaters, reduced_pgar, how = 'inner', on = ['name', 'Year'])

/var/folders/ws/n_fmk_5j2xn553m9_3d3glwc0000gn/T/ipykernel_80558/2179881939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_pgar.rename(columns = {'year_ID':'Year'}, inplace = True)


In [68]:
# creating helper function to calculate weighted statistics
def weighted_avg(series, weights):
    return np.average(series, weights=weights) if weights.sum() > 0 else np.nan

# calculating weighted statistics for all stats from the player gar dataset
def collapse_group(group):
    main_row = group.loc[group['GP'].idxmax()].copy()

    main_row['S%'] = weighted_avg(group['S%'], group['GP'])
    main_row['OFF'] = weighted_avg(group['OFF'], group['GP'])
    main_row['GAR'] = weighted_avg(group['GAR'], group['GP'])
    main_row['avg_pp_toi'] = weighted_avg(group['avg_pp_toi'], group['GP'])
    main_row['GP'] = group['GP'].sum()
    main_row['PPG'] = group['PPG'].sum()

    return main_row
    
collapsed = (
    player_stats
    .groupby(['name', 'Year'], as_index=False)
    .apply(collapse_group, include_groups=False)
    .reset_index(drop=True)
)

In [71]:
# dropping unnecessary columns from dataset and renaming some others
collapsed = collapsed.drop(['GP', 'team_ID', 'situation'], axis = 1)
collapsed.rename(columns = {'team':'Team'}, inplace = True)
player_df  = collapsed.sort_values(['name', 'Year']).reset_index(drop = True)
team_df    = team_stats.rename(columns={'Total GAR': 'TeamGAR'})  

# creating bianry variable that tells us whether a player is on a new team
player_df['prev_team'] = (
    player_df.groupby('name')['Team'].shift(1)
)
player_df['new_team'] = (player_df['Team'] != player_df['prev_team']).astype(int)
player_df['prev_year'] = player_df['Year'] - 1

# Merging previous and current team data(helpful for traded players)
team_prev = (
    team_df.rename(columns={'PP%':'curr_team_pp_prev',
                            'TeamGAR':'curr_team_gar_prev'})
)
player_df = player_df.merge(
    team_prev,
    left_on=['prev_year','Team'],
    right_on=['Year','Team'],
    how='left'
).drop(columns=['Year_y']).rename(columns={'Year_x':'Year'})
team_prev2 = (
    team_df.rename(columns={'Team':'prev_team',
                            'PP%':'prev_team_pp_prev',
                            'TeamGAR':'prev_team_gar_prev'})
)

player_df = player_df.merge(
    team_prev2,
    left_on=['prev_year','prev_team'],
    right_on=['Year','prev_team'],
    how='left'
).drop(columns=['Year_y']).rename(columns={'Year_x':'Year'})

# calculating differences in player and team stats before and after the move if players were moved
player_df['pp_diff']  = np.where(
    player_df['new_team'] == 1,
    player_df['curr_team_pp_prev'] - player_df['prev_team_pp_prev'],
    0
)

player_df['gar_diff'] = np.where(
    player_df['new_team'] == 1,
    player_df['curr_team_gar_prev'] - player_df['prev_team_gar_prev'],
    0
)
player_df.drop(columns = ['prev_year',
                        'curr_team_pp_prev','prev_team_pp_prev',
                        'curr_team_gar_prev','prev_team_gar_prev'],
               inplace = True)
player_df.drop(columns = ['prev_team'], inplace = True)

In [73]:
# creating the training data and shifting statistics one year ahead for the outcome variable
training_df = player_df.fillna(0)
training_df = player_df.sort_values(['name', 'Year']).reset_index(drop = True)
training_df['SeasonGoals'] = (training_df['I_F_goals'] * training_df['icetime'] / 3600).round(0).astype(int)

training_df['Goals_next'] = (
    training_df.groupby('name')['SeasonGoals'].shift(-1)   
)
training_df['icetime_next'] = (
    training_df.groupby('name')['icetime'].shift(-1)   
)
training_df['G_per60_next'] = training_df['Goals_next'] / (training_df['icetime_next'] / 60)

# Creating "finishing skill" metric to evaluate how well a player finishes their opportunities
training_df['fin_skill'] = training_df['SeasonGoals'] - training_df['I_F_xGoals']
training_df['fin_skill_3yr'] = (
    training_df
      .groupby('name')['fin_skill']
      .rolling(3, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)
training_df = training_df[~training_df['Goals_next'].isna()].copy()

In [74]:
# copy of the training data without removing final year of the data(used later for 25-26 predictions)
model_df = player_df.sort_values(['name', 'Year']).reset_index(drop=True)
model_df['SeasonGoals'] = (model_df['I_F_goals'] * model_df['icetime'] / 3600).round(0).astype(int)
# shift each player’s goal total forward one row
model_df['Goals_next'] = (
    model_df.groupby('name')['SeasonGoals'].shift(-1)   # −1 because later row = next season
)
model_df['icetime_next'] = (
    model_df.groupby('name')['icetime'].shift(-1)   # −1 because later row = next season
)
model_df['G_per60_next'] = model_df['Goals_next'] / (model_df['icetime_next'] / 60)
model_df['fin_skill'] = model_df['SeasonGoals'] - model_df['I_F_xGoals']
model_df['fin_skill_3yr'] = (
    model_df
      .groupby('name')['fin_skill']
      .rolling(3, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

In [75]:
# organizing training data and defining variables for the model to take in
rate_cols = ['I_F_xGoals', 'I_F_xRebounds', 'I_F_shotsOnGoal', 'I_F_goals', 'I_F_rebounds',
             'I_F_reboundGoals','I_F_lowDangerShots', 'I_F_mediumDangerShots', 'I_F_highDangerShots',
             'I_F_lowDangerGoals', 'I_F_mediumDangerGoals', 'I_F_highDangerGoals',]  
context   = ['games_played', 'icetime', 'new_team', 'pp_diff', 'gar_diff', 'avg_pp_toi', 'age', 'S%', 'OFF', 'GAR',
            'fin_skill_3yr']
X = training_df[rate_cols + context]
y = training_df['Goals_next']
y_rate = training_df['G_per60_next']
y_toi  = training_df['icetime_next']

In [76]:
# filtering data by year
train_mask  = training_df['Year'] < 2024
X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[~train_mask], y[~train_mask]

In [77]:
# fitting random forest model to find the best fit for predicting goals per 60 rate
model   = RandomForestRegressor(random_state = 42, n_jobs = -1)

param_grid = {
    'n_estimators': [400, 800],
    'max_depth': [None, 15, 25],
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['sqrt', 0.3, 0.6],
}

tscv = TimeSeriesSplit(n_splits=5)  # keeps chronological order

search = GridSearchCV(
    model, param_grid,
    cv = tscv,
    scoring ='neg_root_mean_squared_error',
    verbose=1, n_jobs=-1
).fit(X_train, y_rate[train_mask])
rate_rf = search.best_estimator_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


In [78]:
# fitting gradient boosting quantile model for predicting ice time and using those predictions along
# with goals per 60 predictions from the rf model to make total goal predictions
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
toi_gbq = make_pipeline(
    SimpleImputer(strategy="median"),         
    GradientBoostingRegressor(
        loss='quantile',
        alpha=0.88,
        n_estimators=600,
        max_depth=3,
        learning_rate=0.05,
        random_state=42)
)

mask_24_25   = model_df['Year'] == 2025
features_24_25 = model_df.loc[mask_24_25, rate_cols + context]
toi_gbq.fit(X_train, y_toi[train_mask])
toi_pred = toi_gbq.predict(features_24_25)
g60_pred  = rate_rf.predict(features_24_25)      
total_pred = g60_pred * toi_pred / 60            
latest_pred = (
    pd.DataFrame({
        'name'  : model_df.loc[mask_24_25, 'name'],
        'team'  : model_df.loc[mask_24_25, 'Team'],
        'goals_2026' : total_pred.round(1)
    })
    .sort_values('goals_2026', ascending=False)
)

print(latest_pred.head(15))

                  name team  goals_2026
76       Alex Ovechkin  WSH        43.2
1546  Nathan MacKinnon  COL        39.4
1552       Nazem Kadri  CGY        38.8
1264    Leon Draisaitl  EDM        38.7
198    Auston Matthews  TOR        38.4
1630   Nikita Kucherov  TBL        37.5
1198       Kevin Fiala  LAK        37.4
2039     Tage Thompson  BUF        36.9
732     Filip Forsberg  NSH        36.6
185     Artemi Panarin  NYR        36.3
262      Brady Tkachuk  OTT        36.3
1049      John Tavares  TOR        36.0
881      Jake Guentzel  TBL        35.9
1418   Matthew Tkachuk  FLA        35.8
1394        Matt Boldy  MIN        35.8


g60_pred = rate_rf.predict(X_val)
toi_pred = toi_gbq.predict(X_val)
total_pred = g60_pred * toi_pred / 60

rmse = np.sqrt(mean_squared_error(y_val, total_pred))
print("Rate×TOI RMSE:", rmse)

In [269]:
# Evaluating the model on individual seasons in the data
seasons = sorted(training_df['Year'].unique())

records = []                                   
for cut in seasons[:-1]:                       
    train_mask = training_df['Year'] <= cut
    test_mask  = training_df['Year'] == cut + 1

    # Re‑fit rate + TOI models on data ≤ cut
    rate_rf.fit(X[train_mask],  y_rate[train_mask])
    toi_gbq.fit(X[train_mask], y_toi[train_mask])  

    # Predict totals for season cut+1
    g60  = rate_rf.predict(X[test_mask])
    toi  = toi_gbq.predict(X[test_mask])
    pred = g60 * toi / 60

    # Evaluating rmse and mae
    rmse = np.sqrt(mean_squared_error(y[test_mask], pred))
    mae  = mean_absolute_error(y[test_mask], pred)

    records.append({'season': cut+1, 'rmse': rmse, 'mae': mae})

cv_df = pd.DataFrame(records)
print(cv_df)
print("Mean RMSE:", cv_df['rmse'].mean())

   season      rmse       mae
0    2023  7.302386  5.634878
1    2024  6.914922  5.289626
Mean RMSE: 7.108654194369842


In [270]:
naive = training_df.loc[test_mask, 'SeasonGoals']      # last‑season goals
naive_rmse = np.sqrt(mean_squared_error(y[test_mask], naive))
lift = (naive_rmse - rmse) / naive_rmse * 100
print(f"RMSE lift vs naive: {lift:.1f}%")

RMSE lift vs naive: 5.1%


In [ ]:
# variable importance for goals per 60 model
importances = pd.Series(rate_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print(importances.head(15))

In [ ]:
# variable importance for icetime model
importances = pd.Series(toi_rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print(importances.head(15))